In [ ]:
# default_exp utils.utils

In [ ]:
#export
import os
import random
import torch
import torch.nn as nn
import transformers
import numpy as np
import torch.nn.functional as F

import Hasoc.config as config
import Hasoc.dataset.dataset as dataset

## Some Utility functions

In [ ]:
#export
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

In [ ]:
#export
def create_path(path):
    if not os.path.exists(path):
        path.mkdir(parents=True, exist_ok=True)
    return path

In [ ]:
#export
def create_loader(text, lbls, bs, is_test=False, drop_last=False, sampler=None, ret_dataset=False):
    assert len(text) == len(lbls)
    ds = dataset.BertDataset(text, lbls, is_test=is_test)
    if ret_dataset: return ds
    dl = torch.utils.data.DataLoader(ds, batch_size=bs, drop_last=drop_last,
                                        num_workers=5, sampler=sampler)
    return dl

In [ ]:
#export
class LabelSmoothingCrossEntropy(nn.Module):
    #from fastai
    y_int = True
    def __init__(self, eps:float=0.1, reduction='mean'):
        super().__init__()
        self.eps,self.reduction = eps,reduction

    def forward(self, output, target):
        c = output.size()[-1]
        log_preds = F.log_softmax(output, dim=-1)
        if self.reduction=='sum': loss = -log_preds.sum()
        else:
            loss = -log_preds.sum(dim=-1) #We divide by that size at the return line so sum and not mean
            if self.reduction=='mean':  loss = loss.mean()
        return loss*self.eps/c + (1-self.eps) * F.nll_loss(log_preds, target.long(), reduction=self.reduction)

#     def activation(self, out): return F.softmax(out, dim=-1)
#     def decodes(self, out):    return out.argmax(dim=-1)